In [1]:
import yaml
from google.cloud import bigquery


In [ ]:
config = yaml.load(open("credentials.yml"), yaml.Loader)
client = bigquery.Client.from_service_account_json(config["credentials_path"])


In [2]:
dataset_ref = bigquery.DatasetReference("bigquery-public-data", "san_francisco")
bikeshare_trips_ref = bigquery.TableReference(dataset_ref, "bikeshare_trips")
bikeshare_trips_table = client.get_table(bikeshare_trips_ref)

client.list_rows(bikeshare_trips_table, max_results=5).to_dataframe()


,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


In [6]:
# Query to count the (cumulative) number of trips per day for each date in 2015.
query = """
    WITH trips_by_day AS (
        SELECT DATE(start_date) as trip_date,
               COUNT(1) as num_trips
        FROM `bigquery-public-data.san_francisco.bikeshare_trips`
        WHERE EXTRACT(YEAR FROM start_date) = 2015
        GROUP BY trip_date
    )
    SELECT *,
           SUM(num_trips) OVER (
               ORDER BY trip_date
               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
           ) as cumulative_trips
    FROM trips_by_day
    ORDER BY trip_date
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_results = client.query(query, job_config=safe_config).to_dataframe()
query_results


,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284
...,...,...,...
360,2015-12-27,146,344692
361,2015-12-28,475,345167
362,2015-12-29,502,345669
363,2015-12-30,319,345988


In [20]:
# Tracks the stations where each bike began (in start_station_id)
# and ended (in end_station_id) the day on October 25, 2015 including trip time.

query = """
    SELECT bike_number,
           TIME(start_date) AS start_time,
           TIME(end_date) AS end_time,
           DATE_DIFF(end_date, start_date, MINUTE) AS trip_time_min,
           FIRST_VALUE(start_station_id) OVER (
               PARTITION BY bike_number
               ORDER BY start_date
               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
           ) AS first_station_id,
           LAST_VALUE(end_station_id) OVER (
               PARTITION BY bike_number
               ORDER BY start_date
               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
           ) AS last_station_id,
           start_station_id,
           end_station_id
    FROM `bigquery-public-data.san_francisco.bikeshare_trips`
    WHERE DATE(start_date) = '2015-10-25'
    ORDER BY bike_number ASC
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_results = client.query(query, job_config=safe_config).to_dataframe()
query_results


,bike_number,start_time,end_time,trip_time_min,first_station_id,last_station_id,start_station_id,end_station_id
0,22,13:25:00,13:41:00,16,2,16,2,16
1,25,11:43:00,11:51:00,8,77,51,77,60
2,25,12:14:00,12:20:00,6,77,51,60,51
3,29,14:59:00,15:07:00,8,46,74,46,60
4,29,21:23:00,21:29:00,6,46,74,60,74
...,...,...,...,...,...,...,...,...
290,877,07:17:00,07:23:00,6,54,73,54,51
291,877,08:34:00,08:37:00,3,54,73,51,50
292,877,08:42:00,08:48:00,6,54,73,50,54
293,877,09:23:00,09:27:00,4,54,73,54,56


In [21]:
dataset_ref = bigquery.DatasetReference("bigquery-public-data", "chicago_taxi_trips")
taxi_trips_ref = bigquery.TableReference(dataset_ref, "taxi_trips")
taxi_trips_table = client.get_table(taxi_trips_ref)

client.list_rows(taxi_trips_table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,b3ec3cef49922433e4a5d8c74998af060224e308,e3dfdc6273b53e7bea4294c159fb810c907ffcd2aba9ba...,2015-01-24 21:00:00+00:00,2015-01-24 21:00:00+00:00,240,0.6,17031080100,17031080300,8,8,...,0.0,4.65,Cash,Dispatch Taxi Affiliation,41.907520,-87.626659,POINT (-87.6266589003 41.90752007470001),41.907492,-87.635760,POINT (-87.6357600901 41.9074919303)
1,7ded635e0822f2b6d854744d69caf601d8578fe3,19e804a8eab9224b352e6a384007418b519864a3e7c2d9...,2015-02-04 04:45:00+00:00,2015-02-04 05:00:00+00:00,540,2.9,<NA>,<NA>,22,24,...,3.0,12.25,Cash,Taxi Affiliation Services,41.922761,-87.699155,POINT (-87.69915534320002 41.9227606205),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
2,910e87798340a2b3e1739524f3b2ce9b3e6c62d5,1be31ca6ff375ee0a1a646ffeb90536c2a12453142c482...,2015-03-27 20:30:00+00:00,2015-03-27 20:45:00+00:00,600,1.7,17031081800,17031080201,8,8,...,0.0,7.65,Cash,Taxi Affiliation Services,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.909496,-87.630964,POINT (-87.630963601 41.9094956686)
3,89717b37316bb8a61c27335d56e4b22110b11e16,ea9c7f865233f880e5f00abb728092901eeaf52c85a8c1...,2015-01-29 16:00:00+00:00,2015-01-29 16:15:00+00:00,240,0.0,17031320400,17031081403,32,8,...,0.0,5.85,Cash,Choice Taxi Association,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259)
4,9c7429d65b397e01c79d8cc739cc3abaeac6ab84,0d61c4f9c8cb8d280fce388789bd7e26d2a692c0e510ca...,2014-12-13 15:30:00+00:00,2014-12-13 15:45:00+00:00,1200,2.7,17031320400,17031081700,32,8,...,1.0,11.45,Cash,Northwest Management LLC,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365)


In [30]:
# Amend the query below to return a DataFrame with two columns:
# - `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
# - `avg_num_trips` - shows the average number of daily trips, calculated over a window including
# the value for the current date, along with the values for the preceding 3 days and the following 3 days,
# as long as the days fit within the three-month time frame.  For instance, when calculating the value
# in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days,
# the current date, and the following 3 days.

query = """
    WITH trips_by_day AS (
        SELECT DATE(trip_start_timestamp) AS trip_date,
        COUNT (1) as num_trips
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
        WHERE trip_start_timestamp >= '2016-01-01' AND
              trip_start_timestamp < '2016-04-01'
        GROUP BY trip_date
    )
    SELECT trip_date, 
           num_trips,
           AVG(num_trips) OVER (
               ORDER BY trip_date
               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
           ) AS avg_num_trips
    FROM trips_by_day
    ORDER BY trip_date
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_results = client.query(query, job_config=safe_config).to_dataframe()
query_results


,trip_date,num_trips,avg_num_trips
0,2016-01-01,97202,73313.750000
1,2016-01-02,69000,72998.200000
2,2016-01-03,59286,73452.166667
3,2016-01-04,67767,74401.428571
4,2016-01-05,71736,73767.285714
...,...,...,...
86,2016-03-27,65311,84715.000000
87,2016-03-28,73250,78639.428571
88,2016-03-29,78059,75684.000000
89,2016-03-30,89492,74274.400000


In [42]:
# Three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.
# Amend the query to return an additional column called `trip_number` which shows the order in which the trips
# were taken from their respective community areas.  So, the first trip of the day originating from community
# area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.
# Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

query = """
    SELECT pickup_community_area,
           trip_start_timestamp,
           trip_end_timestamp,
           RANK() OVER (
               PARTITION BY pickup_community_area
               ORDER BY trip_start_timestamp ASC
           ) AS trip_number
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE DATE(trip_start_timestamp) = '2013-10-03'
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_results = client.query(query, job_config=safe_config).to_dataframe()
query_results


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,44,2013-10-03 03:45:00+00:00,2013-10-03 03:45:00+00:00,1
1,44,2013-10-03 06:15:00+00:00,2013-10-03 06:15:00+00:00,2
2,44,2013-10-03 07:15:00+00:00,2013-10-03 07:15:00+00:00,3
3,44,2013-10-03 07:30:00+00:00,2013-10-03 08:00:00+00:00,4
4,44,2013-10-03 08:45:00+00:00,2013-10-03 09:15:00+00:00,5
...,...,...,...,...
84263,69,2013-10-03 12:00:00+00:00,2013-10-03 12:15:00+00:00,5
84264,69,2013-10-03 12:00:00+00:00,2013-10-03 12:00:00+00:00,5
84265,69,2013-10-03 15:15:00+00:00,2013-10-03 15:30:00+00:00,8
84266,69,2013-10-03 18:30:00+00:00,2013-10-03 18:30:00+00:00,9


In [56]:
# The (partial) query in the code cell below shows, for each trip in the selected time frame,
# the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`.
# Your task in this exercise is to edit the query to include an additional `prev_break` column
# that shows the length of the break (in minutes) that the driver had before each trip started
# (this corresponds to the time between `trip_start_timestamp` of the current trip and
# `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`,
# and order the results within each partition by `trip_start_timestamp`.

query = """
    SELECT taxi_id,
           trip_start_timestamp,
           trip_end_timestamp,
           DATE_DIFF(
               trip_start_timestamp,
               LAG(trip_end_timestamp) OVER (
                   PARTITION BY taxi_id
                   ORDER BY trip_start_timestamp
               ),
               MINUTE
           ) AS prev_break
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE DATE(trip_start_timestamp) = '2013-10-03'
    ORDER BY taxi_id, trip_start_timestamp
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=40 * 10**10)
query_results = client.query(query, job_config=safe_config).to_dataframe()
query_results


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:15:00+00:00,2013-10-03 11:30:00+00:00,<NA>
1,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:30:00+00:00,2013-10-03 11:45:00+00:00,0
2,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:45:00+00:00,2013-10-03 12:00:00+00:00,0
3,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:00:00+00:00,2013-10-03 12:00:00+00:00,0
4,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:15:00+00:00,2013-10-03 12:15:00+00:00,15
...,...,...,...,...
84263,ffef3b7e2f2b7b6d5eef07fd1dfa3cc0eb426d7c225520...,2013-10-03 15:15:00+00:00,2013-10-03 13:00:00+00:00,<NA>
84264,fff3279af11dfade5e308184a5cc6c1ca2d33764634d96...,2013-10-03 09:45:00+00:00,2013-10-03 09:30:00+00:00,<NA>
84265,fff3279af11dfade5e308184a5cc6c1ca2d33764634d96...,2013-10-03 11:45:00+00:00,2013-10-03 11:45:00+00:00,135
84266,fff3279af11dfade5e308184a5cc6c1ca2d33764634d96...,2013-10-03 13:00:00+00:00,2013-10-03 13:00:00+00:00,75
